In [1]:
import nltk
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.decomposition import NMF, TruncatedSVD
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv('products_sentiment_train.tsv',delimiter='\t',header=None)
train_data.columns = ['X','y']
X = train_data.X.values # (2000,)
y = train_data.y.values # (2000,)

In [3]:
def text_classifier(vectorizer, transformer, classifier, cv=10):
    pipeline = Pipeline(
            [("vectorizer", vectorizer),
            ("transformer", transformer),
            ("classifier", classifier)]
        )
    pipeline.fit(X,y)
    accuracy = cross_val_score(pipeline, X, y, cv=cv).mean()
    return accuracy

In [4]:
# самый простой вариант
text_classifier(CountVectorizer(),None,LogisticRegression())

0.7724854746368659

In [5]:
# стоп слова не делают лучше
text_classifier(CountVectorizer(stop_words = stopwords.words('english')),None,LogisticRegression())

0.7529726868171704

In [6]:
# биграммы делают лучше,  максимальная точность 0.7720104002600066 с LogisticRegression
for clf in [LogisticRegression(), LinearSVC(), SGDClassifier()]:
    print(
        text_classifier(CountVectorizer(ngram_range=(1,2)),None,clf)
    )

0.7720104002600066
0.7690103752593815


C:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\stoc

0.7634851496287407


C:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [7]:
# Tfidf преобразование улучшает. максимальная точность 0.7854481862046551 с LinearSVC
for clf in [LogisticRegression(), LinearSVC(), SGDClassifier()]:
    print(
        text_classifier(CountVectorizer(ngram_range=(1,2)),TfidfTransformer(),clf)
    )

0.7495074376859422
0.7854481862046551


C:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\stoc

0.7869781119527989


C:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [8]:
# увеличиваем нграммы! получаем наилучший результат ngram_range=(1,4)
for ngram in range(2,8,1):
    print(ngram, 
          text_classifier(CountVectorizer(ngram_range=(1,ngram)),TfidfTransformer(),LinearSVC())
         )

2 0.7854481862046551
3 0.7869607240181005
4 0.7884757243931099
5 0.785983187079677
6 0.783985649641241
7 0.7834931373284332


In [9]:
# нграммы по буквам пока что лучшие
text_classifier(CountVectorizer(ngram_range=(2,10), min_df=1, analyzer='char_wb'),TfidfTransformer(),LinearSVC())

0.7814731618290457

In [11]:
# нграммы по буквам еще лучше, поочереди подбирал параметры ngram_range. Лучшая точность с ngram_range=(4,9)
text_classifier(CountVectorizer(ngram_range=(4,9), analyzer='char'),TfidfTransformer(),LinearSVC())

0.8015110002750069

In [12]:
# попробуем понижение размерности пространства признаков частот слов - лучше не стало
text_classifier(TfidfVectorizer(), TruncatedSVD(n_components=100), LinearSVC()) 

0.7690003250081252

In [13]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [14]:
# пробуем решающие деревья - фигня
text_classifier(CountVectorizer(), TruncatedSVD(n_components=50), RandomForestClassifier(100))

0.7020464886622165

In [15]:
test_data = pd.read_csv('products_sentiment_test.tsv',delimiter='\t') # shape (500, 2)
test_data.head()

Id                                               text
0   0  so , why the small digital elph , rather than ...
1   1  3/4 way through the first disk we played on it...
2   2  better for the zen micro is outlook compatibil...
3   3    6 . play gameboy color games on it with goboy .
4   4  likewise , i 've heard norton 2004 professiona...

In [18]:
def text_classifier_predict(vectorizer, transformer, classifier, X_train):
    pipeline = Pipeline(
            [("vectorizer", vectorizer),
            ("transformer", transformer),
            ("classifier", classifier)]
        )
    pipeline.fit(X,y)
    return pipeline.predict(X_train)

In [19]:
y_test1 = text_classifier_predict(CountVectorizer(ngram_range=(1,2)),None,LogisticRegression(),
                                 test_data.text.values)
with open('answer1.txt',mode='w') as ff:
    for i in range(500):
        ff.write(str(i)+','+str(y_test1[i])+'\n')

In [20]:
y_test2 = text_classifier_predict(CountVectorizer(ngram_range=(4,9), analyzer='char'),TfidfTransformer(),LinearSVC(),
                                 test_data.text.values)
with open('answer2.txt',mode='w') as ff:
    for i in range(500):
        ff.write(str(i)+','+str(y_test2[i])+'\n')

In [21]:
y_test3 = text_classifier_predict(CountVectorizer(ngram_range=(1,4)),TfidfTransformer(),LinearSVC(),
                                 test_data.text.values)
with open('answer3.txt',mode='w') as ff:
    for i in range(500):
        ff.write(str(i)+','+str(y_test3[i])+'\n')

Я попробовал все эти три топовых метода, везде счет 0.8150 или чуть ниже